## Labeling images

In [1]:
!pip install --upgrade pyqt5 lxml

In [2]:
!mkdir labelImg
!git clone https://github.com/tzutalin/labelImg labelImg

Cloning into 'labelImg'...


In [2]:
!cd labelImg && pyrcc5 -o libs/resources.py resources.qrc

In [3]:
!cd labelImg && python labelImg.py

## Extract text from xml

In [4]:
import os
from glob import glob
import pandas as pd
from functools import reduce
from xml.etree import ElementTree as et

In [5]:
# load all xml files and store in a list
xml_list = glob('./data_images/*.xml')

# replace \\ with /
xml_list = list(map(lambda x: x.replace('\\','/'), xml_list))

In [6]:
def extract_text(filename):
    # read xml files
    tree = et.parse(filename)
    root = tree.getroot()

    # extract filename
    image_name = root.find('filename').text

    # width and height of the image
    width = root.find('size').find('width').text
    height = root.find('size').find('height').text

    objs = root.findall('object')
    parser = []
    for obj in objs:
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = bndbox.find('xmin').text
        xmax = bndbox.find('xmax').text
        ymin = bndbox.find('ymin').text
        ymax = bndbox.find('ymax').text
        parser.append([image_name, width, height, name, xmin, xmax, ymin, ymax])
    
    return parser

In [7]:
parser_all = list(map(extract_text, xml_list))

In [8]:
data = reduce(lambda x, y: x + y, parser_all)

In [9]:
df = pd.DataFrame(data, columns = ['filename', 'width', 'height', 'name', 'xmin', 'xmax', 'ymin', 'ymax'])

In [10]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax
0,alvan-nee-1VgfQdCuX-4-unsplash.jpg,1851,2780,dog,371,903,1317,2603
1,alvan-nee-1VgfQdCuX-4-unsplash.jpg,1851,2780,dog,862,1539,1226,2594
2,alvan-nee-eoqnr8ikwFE-unsplash.jpg,2744,4049,dog,398,2258,1024,3664
3,alvan-nee-T-0EW-SEbsE-unsplash.jpg,5784,3861,dog,1152,2538,817,3277
4,alvan-nee-T-0EW-SEbsE-unsplash.jpg,5784,3861,dog,2878,4705,484,3861


In [11]:
df['name'].value_counts()

name
person    17
dog       11
bus        7
car        7
bike       7
plane      4
sofa       3
chair      2
Name: count, dtype: int64

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  58 non-null     object
 1   width     58 non-null     object
 2   height    58 non-null     object
 3   name      58 non-null     object
 4   xmin      58 non-null     object
 5   xmax      58 non-null     object
 6   ymin      58 non-null     object
 7   ymax      58 non-null     object
dtypes: object(8)
memory usage: 3.8+ KB


In [13]:
# type conversion
cols = ['width', 'height', 'xmin', 'xmax', 'ymin', 'ymax']
df[cols] = df[cols].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  58 non-null     object
 1   width     58 non-null     int64 
 2   height    58 non-null     int64 
 3   name      58 non-null     object
 4   xmin      58 non-null     int64 
 5   xmax      58 non-null     int64 
 6   ymin      58 non-null     int64 
 7   ymax      58 non-null     int64 
dtypes: int64(6), object(2)
memory usage: 3.8+ KB


In [14]:
# center x, center y
df['center_x'] = ((df['xmax'] + df['xmin']) / 2) / df['width']
df['center_y'] = ((df['ymax'] + df['ymin']) / 2) / df['height']

# width, height
df['w'] = (df['xmax'] - df['xmin']) / df['width']
df['h'] = (df['ymax'] - df['ymin']) / df['height']

In [15]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,alvan-nee-1VgfQdCuX-4-unsplash.jpg,1851,2780,dog,371,903,1317,2603,0.344138,0.705036,0.287412,0.462590
1,alvan-nee-1VgfQdCuX-4-unsplash.jpg,1851,2780,dog,862,1539,1226,2594,0.648568,0.687050,0.365748,0.492086
2,alvan-nee-eoqnr8ikwFE-unsplash.jpg,2744,4049,dog,398,2258,1024,3664,0.483965,0.578908,0.677843,0.652013
3,alvan-nee-T-0EW-SEbsE-unsplash.jpg,5784,3861,dog,1152,2538,817,3277,0.318983,0.530174,0.239627,0.637141
4,alvan-nee-T-0EW-SEbsE-unsplash.jpg,5784,3861,dog,2878,4705,484,3861,0.655515,0.562678,0.315871,0.874644


## Split data

In [16]:
unique_files = df['filename'].unique()

In [18]:
from sklearn.model_selection import train_test_split

train_name, test_name = train_test_split(unique_files, test_size = 0.2, random_state=42)

In [19]:
train_df = df[df['filename'].isin(train_name)]
test_df = df[df['filename'].isin(test_name)]

## Assign ID number to objects

In [20]:
# label encoding
def label_encoding(x):
    labels = {
        "person": 0,
        "car" : 1,
        "chair": 2,
        "bottle": 3,
        "pottedplant": 4,
        "bird": 5,
        "dog": 6,
        "sofa": 7,
        "bike": 8,
        "horse": 9,
        "boat": 10,
        "motorbike": 11,
        "cat": 12,
        "television": 13,
        "cow": 14,
        "sheep": 15,
        "plane": 16,
        "train": 17,
        "table": 18,
        "bus":19
    }
    return labels[x]

In [21]:
train_df['id'] = train_df['name'].apply(label_encoding)
test_df['id'] = test_df['name'].apply(label_encoding)

C:\Users\apo09\AppData\Local\Temp\ipykernel_13088\3116627190.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['id'] = train_df['name'].apply(label_encoding)
C:\Users\apo09\AppData\Local\Temp\ipykernel_13088\3116627190.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['id'] = test_df['name'].apply(label_encoding)


In [22]:
train_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h,id
0,alvan-nee-1VgfQdCuX-4-unsplash.jpg,1851,2780,dog,371,903,1317,2603,0.344138,0.705036,0.287412,0.462590,6
1,alvan-nee-1VgfQdCuX-4-unsplash.jpg,1851,2780,dog,862,1539,1226,2594,0.648568,0.687050,0.365748,0.492086,6
2,alvan-nee-eoqnr8ikwFE-unsplash.jpg,2744,4049,dog,398,2258,1024,3664,0.483965,0.578908,0.677843,0.652013,6
3,alvan-nee-T-0EW-SEbsE-unsplash.jpg,5784,3861,dog,1152,2538,817,3277,0.318983,0.530174,0.239627,0.637141,6
4,alvan-nee-T-0EW-SEbsE-unsplash.jpg,5784,3861,dog,2878,4705,484,3861,0.655515,0.562678,0.315871,0.874644,6


## Save Image and Labels in Text

In [41]:
import os
from shutil import move

In [ ]:
train_folder = 'data_images/train'
test_folder = 'data_images/test'

os.mkdir(train_folder)
os.mkdir(test_folder)

In [51]:
cols = ['filename', 'id', 'center_x', 'center_y', 'w', 'h']
groupby_obj_train = train_df[cols].groupby('filename')
groupby_obj_test  = test_df[cols].groupby('filename')

In [ ]:
def save_data(filename, folder_path, group_obj):
    # move images
    src = os.path.join('images', filename)
    dst = os.path.join(folder_path, filename)
    move(src, dst)

    # save the labels
    text_filename = os.path.join(folder_path, 
                                 os.path.splitext(filename)[0] + '.txt')
    group_obj.get_group(filename).set_index('filename').to_csv(text_filename, index=False, sep=' ', header=False)

In [ ]:
filename_series_train = pd.Series(groupby_obj_train.groups.keys())
filename_series_train.apply(save_data, args=(train_folder, groupby_obj_train))

In [56]:
filename_series_test = pd.Series(groupby_obj_test.groups.keys())
filename_series_test.apply(save_data, args=(test_folder, groupby_obj_test))

0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
dtype: object